# Dashboard Visualizations

This notebook creates interactive visualizations for the financial dashboard.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sys
import os

# Add src directory to path to import custom modules
sys.path.append(os.path.join('..', 'src'))

from data_loading import load_financial_dataset
from data_cleaning import clean_financial_data
from feature_engineering import engineer_features
from visualization import (
    plot_candlestick, plot_correlation_heatmap, plot_time_series,
    plot_stock_comparison, plot_returns_distribution, create_interactive_dashboard
)

# Load and process the financial dataset
df = load_financial_dataset()
df_clean = clean_financial_data(df)
df_featured = engineer_features(df_clean)

print(f'Dataset shape: {df_featured.shape}')
print(f'Date range: {df_featured["Date"].min()} to {df_featured["Date"].max()}')
print(f'Stock indices: {df_featured["Stock Index"].unique()}')

In [ ]:
# Create interactive stock price comparison
print('Creating stock price comparison...')

# Focus on a specific stock for detailed visualization
selected_stock = 'S&P 500'  # You can change this to any stock in your dataset
df_stock = df_featured[df_featured['Stock Index'] == selected_stock].copy()

# Plot candlestick chart for the selected stock
print(f'Creating candlestick chart for {selected_stock}...')
plot_candlestick(df_stock, stock=selected_stock)

In [ ]:
# Create technical indicators visualization
print('Creating technical indicators visualization...')

# Create time series plots for key technical indicators
tech_cols = ['Close Price', 'Close Price_MA_20', 'Close Price_MA_50', 'Close Price_RSI']
available_tech_cols = [col for col in tech_cols if col in df_stock.columns]

if available_tech_cols:
    plot_time_series(df_stock, available_tech_cols, stock=selected_stock)

# Plot returns distribution
if 'Close Price_return' in df_stock.columns:
    plot_returns_distribution(df_stock, 'Close Price_return')

In [ ]:
# Create comprehensive correlation heatmap
print('Creating correlation heatmap...')

# Select important financial features for correlation analysis
important_cols = [
    'Open Price', 'Close Price', 'Daily High', 'Daily Low', 'Trading Volume',
    'Close Price_return', 'Close Price_volatility', 'Close Price_RSI',
    'GDP Growth (%)', 'Inflation Rate (%)', 'Unemployment Rate (%)', 'Interest Rate (%)',
    'Consumer Confidence Index', 'Gold Price (USD per Ounce)', 'Crude Oil Price (USD per Barrel)'
]

available_cols = [col for col in important_cols if col in df_stock.columns]
correlation_matrix = plot_correlation_heatmap(df_stock, cols=available_cols, 
                                              figsize=(12, 10), annot=True)

In [ ]:
# Create an interactive dashboard with multiple visualizations
print('Creating interactive dashboard...')

# Create the dashboard with the selected stock data
dashboard = create_interactive_dashboard(df_stock)
dashboard.show()

# Save the dashboard as an HTML file
dashboard.write_html('../reports/interactive_dashboard.html')
print('Interactive dashboard saved to ../reports/interactive_dashboard.html')

In [ ]:
# Create additional visualizations for all stocks
print('Creating additional visualizations...')

# Plot stock comparison for all available stocks (first 1000 rows for performance)
plot_stock_comparison(df_clean.head(1000))

# Create a subplot with multiple visualizations
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Price Trends', 'Trading Volume', 'Returns Distribution', 'Price vs Volume'),
    specs=[[{"secondary_y": False}, {"secondary_y": False}],
           [{"secondary_y": False}, {"secondary_y": False}]]
)

# Add price trend
fig.add_trace(
    go.Scatter(x=df_stock['Date'], y=df_stock['Close Price'], 
               name='Close Price', line=dict(color='blue')),
    row=1, col=1
)

# Add trading volume
if 'Trading Volume' in df_stock.columns:
    fig.add_trace(
        go.Bar(x=df_stock['Date'], y=df_stock['Trading Volume'], 
               name='Trading Volume', marker_color='green', opacity=0.6),
        row=1, col=2
    )

# Add returns distribution
if 'Close Price_return' in df_stock.columns:
    returns = df_stock['Close Price_return'].dropna()
    fig.add_trace(
        go.Histogram(x=returns, name='Returns Distribution', marker_color='orange', opacity=0.7),
        row=2, col=1
    )

# Add price vs volume scatter
if 'Trading Volume' in df_stock.columns:
    fig.add_trace(
        go.Scatter(x=df_stock['Trading Volume'], y=df_stock['Close Price'], 
                   mode='markers', name='Price vs Volume', marker=dict(size=5, opacity=0.6)),
        row=2, col=2
    )

# Update layout
fig.update_layout(height=800, showlegend=True, 
                  title_text=f"Comprehensive Dashboard for {selected_stock}")

# Show the figure
fig.show()

print('Visualizations created!')